## Récupération des corpus Capitains

In [1]:
CHECKSUM_PATH = "./checksum-xmls.csv"
CORPORA_PATH = "../../data/raw/corpora/**/*"
WRITE_TEXT = True # Set to True for really applying XSL
FORCE_REGENERATE = False  # Mostly useful if the XSL changes
FORCE_GENERATE_CSV_FROM_SCRATCH = False # Force writing from scratch the CHECKSUM

import tqdm
import glob
from MyCapytain.resolvers.cts.local import CtsCapitainsLocalResolver


repositories = list(glob.glob(CORPORA_PATH, recursive=False))
resolver = CtsCapitainsLocalResolver(repositories)

../../data/raw/corpora/lascivaroma_additional-texts/lascivaroma_additional-texts/data/phi1351/phi005/phi1351.phi005.perseus-eng1.xml is not present
../../data/raw/corpora/lascivaroma_priapeia/lascivaroma_priapeia/data/phi1103/phi001/phi1103.phi001.lascivaroma-eng1.xml is not present
../../data/raw/corpora/lascivaroma_priapeia/lascivaroma_priapeia/data/phi1103/phi001/phi1103.phi001.lascivaroma-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0472/phi001/phi0472.phi001.perseus-eng3.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0472/phi001/phi0472.phi001.perseus-eng4.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0448/phi001/phi0448.phi001.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0448/phi002/phi0448.phi002.perseus-eng2.xml is 

../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0690/phi003/phi0690.phi003.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi1017/phi011/phi1017.phi011.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0660/phi003/phi0660.phi003.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi1002/phi001/phi1002.phi001.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0914/phi001/phi0914.phi001.perseus-eng3.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0917/phi001/phi0917.phi001.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0119/phi004/phi0119.phi

../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi1351/phi004/phi1351.phi004.perseus-eng1.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi1351/phi001/phi1351.phi001.perseus-eng2.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi1351/phi001/phi1351.phi001.perseus-eng1.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi1351/phi002/phi1351.phi002.perseus-eng1.xml is not present
../../data/raw/corpora/PerseusDL_canonical-latinLit/PerseusDL_canonical-latinLit/data/phi0550/phi001/phi0550.phi001.perseus-eng1.xml is not present


## Récupération des métadonnées

In [2]:
import pandas

metadata = pandas.read_csv("../../data/raw/datation.tsv", dialect="excel-tab").fillna("")
authors = {
    ".".join(text["URN"].split(".")[:1]): text
    for _, text in metadata.iterrows()
}

## Données absentes

In [3]:
texts = {
    str(text.id): text 
    for text in resolver.getMetadata().readableDescendants
    if text.lang == "lat"
}

not_found = set(texts.keys()).difference(set(metadata["URN"].to_list()))

cols = """Vérifié 	Ignore 	
        Nom FR 	Birth 	Death 	
        certainty 	certainty.1 	
        Citation level 	Name of citation level 	
        Sources 	Sources Pages 	Notes"""

new_elements = []

print(not_found)

for nf in list(sorted(not_found)):
    identifier = nf
    citation_level = ",".join([str(cite.name) for cite in list(texts[nf].citation)])
    author = texts[nf].parent.parent.get_label()
    author_id = str(texts[nf].parent.parent.id)
    # print(author_id)
    work = texts[nf].parent.get_label()
    temp = {
        "URN": nf,
        "Nom EN": author,
        "Name of citation level": citation_level,
        "Titre": work
    }
    new_elements.append({
        key: temp.get(key, "")
        for key in metadata.columns
    })
    new_elements[-1].update(temp)
    if author_id in authors:
        new_elements[-1].update(
            authors[author_id][[
                "Birth", "Death", "Sources", "Sources Pages", "Notes", "Nom FR",
                "certainty", "certainty.1"]
            ].to_dict())
    
new = pandas.DataFrame(new_elements)
new.to_csv("to_be_edited.csv")

set()


In [4]:
from IPython.display import display, HTML
display(HTML(new.to_html()))

""


In [5]:
#from MyCapytain.common.constants import Mimetypes
##print(resolver.getTextualNode("urn:cts:latinLit:stoa0024.stoa001.digilibLT-lat1").export(Mimetypes.XML.TEI))